## Import

https://seong6496.tistory.com/169

In [1]:
from tqdm import tqdm

import numpy as np
import pandas as pd
import re


In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Korea_Investment_and_Securities\\scripts\\ipynb'

* etf com crawling

In [4]:
from bs4 import BeautifulSoup

import requests
import pickle

In [5]:
agent = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

In [10]:
## HTTP GET Request
req = requests.get('https://www.etf.com/AIEQ#efficiency',
              headers=agent)

## HTTP GET Request
#req = requests.get('https://www.etf.com/AIEQ#efficiency')
## HTML 소스 가져오기
html = req.text
## HTTP Header 가져오기
header = req.headers
## HTTP Status 가져오기 (200: 정상)
status = req.status_code
## HTTP가 정상적으로 되었는지 (True/False)
is_ok = req.ok

In [11]:
status

503

In [ ]:
#for idx, sym in tqdm(enumerate(tot_df.Symbol[:5])):
for idx, sym in tqdm(enumerate(df_theme.종목코드)):
    ## HTTP GET Request
    req = requests.get('https://finance.yahoo.com/quote/{code}/profile?p={code}'.format(code=sym),
                  headers=agent)
    ## HTML 소스 가져오기
    html = req.text
    ## HTTP Header 가져오기
    header = req.headers
    ## HTTP Status 가져오기 (200: 정상)
    status = req.status_code
    ## HTTP가 정상적으로 되었는지 (True/False)
    is_ok = req.ok
    
    '''
    if "Please check your spelling. Try our suggested matches or see results in other tabs." in html:
        continue
    '''
    
    soup = BeautifulSoup(html, 'html.parser')
    try:
        description = soup.select('section > p')[0]
    except IndexError:
        continue
    desc_dict[sym] = description.text
    